# Home Depot Product Search Relevance IRDM Porject

## <font color='green'>Setup 1</font>: Envrionment Setup

In [4]:
import warnings
warnings.filterwarnings('ignore')

from config import *

import time
start_time = time.time()

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.mpl_style = 'default'

import re
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor,AdaBoostRegressor,RandomForestClassifier,AdaBoostClassifier
from collections import Counter

stemmer = SnowballStemmer('english')

## <font color='green'>Setup 2</font>: Import Data

In [5]:
df_all = pd.read_csv('../IRDM/all_data.csv') #add labelencoder

In [6]:
df_all.head()

,group id,id,product_uid,relevance,is_query_misspelled,len_of_query,len_of_title,word_in_title_num,word_in_title_sum,word_in_title_let,word_in_title_numratio,word_in_title_letratio,word_in_description_num,word_in_description_sum,word_in_description_let,word_in_description_numratio,word_in_description_letratio,brandinquery_brand_matching,brandinquery_brandintitile_matching,brandinquery_bullets_brand_matching,brandinquery_decription_brand_matching,materialinquery_material_matching,materialinquery_materialintitile_matching,materialinquery_bullets_material_matching,materialinquery_decription_material_matching,word2vec_product_title_1,word2vec_product_desc_1,word2vec_bullet_1,word2vec_all_1,word2vec_product_title_2,word2vec_product_desc_2,word2vec_bullet_2,word2vec_all_2,cv_product_title_sim,cv_product_desc_sim,cv_bullet_sim,tfidf_product_title_sim,tfidf_product_desc_sim,tfidf_bullet_sim,jaccard_product_title,jaccard_product_desc,seqmatch_title_ratio,seqmatch_title_ratioscaled,seqmatch_description_ratio,seqmatch_description_ratioscaled,seqmatch_bullets_ratio,seqmatch_bullets_ratioscaled,seqmatch_desc&bullets_ratio,seqmatch_desc&bullets_ratioscaled
0,0,104248,133315,2.00,1.0,3,7,1,1,3,0.333333,0.375,2,4,7,0.666667,0.875,0,0,0,0,0,0,0,0,0.703518,0.684985,0.634746,0.681723,0.668858,0.680586,0.630581,0.671879,0.408248,0.273434,0.000000,0.263700,0.256759,0.000000,0.111111,0.035088,0.230769,0.969231,0.038217,1.761785,0.006006,0.193994,0.022641,1.777358
1,0,85412,125403,2.33,1.0,3,4,2,2,7,0.666667,0.875,2,8,7,0.666667,0.875,0,0,0,0,0,0,0,0,0.768808,0.669086,0.542441,0.628205,0.760264,0.681578,0.585099,0.650896,0.707107,0.546869,0.421637,0.687739,0.405643,0.246514,0.400000,0.026667,0.642857,1.157143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,46280,111472,2.00,1.0,3,5,0,0,0,0.000000,0.000,1,1,4,0.333333,0.500,0,0,0,0,0,0,0,0,0.552219,0.624677,0.581307,0.622914,0.504969,0.607570,0.560937,0.600052,0.000000,0.085749,0.176777,0.000000,0.044307,0.089601,0.000000,0.025641,0.256410,0.743590,0.005602,0.194396,0.121622,1.678378,0.004032,0.195970
3,0,210176,197110,1.33,1.0,3,8,1,1,4,0.333333,0.500,1,4,4,0.333333,0.500,0,0,0,0,0,0,0,0,0.615612,0.655342,0.596934,0.650199,0.567920,0.662082,0.609637,0.653505,0.500000,0.261488,0.000000,0.205782,0.161984,0.000000,0.111111,0.014286,0.235294,0.964706,0.003630,0.196372,0.000000,0.000000,0.002587,0.197411
4,0,2808,100490,3.00,1.0,3,5,2,2,7,0.666667,0.875,2,10,7,0.666667,0.875,0,0,0,0,0,0,0,0,0.776918,0.700988,0.000000,0.711420,0.755178,0.681607,0.000000,0.690727,0.707107,0.439375,0.000000,0.637174,0.280001,0.000000,0.333333,0.024096,0.486486,1.313514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
PROCESSINGTEXT_DIR='../IRDM/processing_text'
df_train_pre = pd.read_csv(PROCESSINGTEXT_DIR + '/df_train_processed-pairwise.csv', encoding="ISO-8859-1")
df_test = pd.read_csv(PROCESSINGTEXT_DIR + '/df_test_processed-pairwise.csv', encoding="ISO-8859-1")
# df_attr_bullet = pd.read_csv(PROCESSINGTEXT_DIR + '/df_attribute_bullets_processed.csv')
# df_pro_desc = pd.read_csv(PROCESSINGTEXT_DIR + '/df_product_descriptions_processed.csv') 

num_train = df_train_pre.shape[0]
num_test = df_test.shape[0]


In [8]:
train = df_all.iloc[:num_train]
test = df_all.iloc[num_train:]
id_test = df_test['id']

In [9]:
len(train),len(test)

(65517, 88373)

## <font color='green'>Setup 3</font>: Preprocessing

In [10]:
import itertools
def preprocessing_data(data):
    index=[]
    X_new = []
    y_new = []
    name_new=[]
    for m in list(set(data['group id'])):
        a = data[data['group id']==m]
        X_data = a.drop(['product_uid','id','relevance','group id','len_of_query'],axis=1).values
        y_data= a['relevance'].values
        pid= a['product_uid'].values

        comb = itertools.combinations(range(X_data.shape[0]), 2)
        for k, (i, j) in enumerate(comb):
            if y_data[i] == y_data[j]: # skip if same target or different group
                continue
            X_new.append(X_data[i] - X_data[j])
            y_new.append(np.sign(y_data[i] - y_data[j]))
            name_new.append(str(int(pid[i]))+"-"+str(int(pid[j])))
            index.append(m)
    return X_new,y_new,name_new,index

In [11]:
def data_frame_made(X_new,y_new,name_new,index):
    df_svm = pd.DataFrame({"binary":y_new, "feature":X_new,"document group":name_new,"searchterm id":index}, columns=['searchterm id','document group','feature','binary'])
    splitted = df_svm['feature'].apply(lambda x: pd.Series(list(x)))
    splitted.columns = ['feature'+str(x) for x in splitted.columns]
    df_train = pd.concat([df_svm, splitted], axis=1)
    return df_svm,splitted,df_train

In [12]:
X_train,y_train,name_train,index_train= preprocessing_data(train)
df_svm_train,splitted_train,df_train = data_frame_made(X_train,y_train,name_train,index_train)
X_test,y_test,name_test,index_test= preprocessing_data(test)
df_svm_test,splitted_test,df_test = data_frame_made(X_test,y_test,name_test,index_test)
# df_svm_train = df_svm_train.drop('feature',axis=1)
# df_svm_test = df_svm_test.drop('feature',axis=1)

## <font color='green'>Setup 4</font>: SVM Model

In [13]:
from sklearn import svm
t_1 = time.time()
clf_svm = svm.SVC()
clf_svm.fit(splitted_train,np.round(df_svm_train.binary))
'model training time:',round((time.time()-t_1)/60,1) ,'minutes\n'#around 86 min

('model training time:', 86.4, 'minutes\n')

In [14]:
y_pred_train = clf_svm.predict(splitted_train)
y_pred_test = clf_svm.predict(splitted_test)
svm_train = pd.DataFrame({"searchterm id":df_svm_train['searchterm id'],"document group":df_svm_train['document group'], "binary":df_svm_train.binary,"bi_predict":y_pred_train}, columns=["searchterm id","document group", "binary","bi_predict"])
svm_train.to_csv("svm_train.csv", index = False)
svm_test = pd.DataFrame({"searchterm id":df_svm_test['searchterm id'],"document group":df_svm_test['document group'], "binary":df_svm_test.binary,"bi_predict":y_pred_test}, columns=["searchterm id","document group", "binary","bi_predict"])
svm_test.to_csv("svm_test.csv", index = False)

In [15]:
svm_train = pd.read_csv("../IRDM/svm_train.csv")
svm_test = pd.read_csv("../IRDM/svm_test.csv")

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_svm_train.binary, svm_train.bi_predict)
cm = cm.astype('float') / cm.sum(axis=1)
cm#[true negatives, false positives], [false negatives,true positives]

array([[ 0.67247188,  0.33127878],
       [ 0.48265301,  0.51181994]])

## <font color='green'>Setup 5</font>: Ranking/Evaluation

In [17]:
svm_test['win_doc']=svm_test['document group'].map(lambda x: x.split('-')[1])
svm_test['win_doc'][svm_test.bi_predict> 0]=svm_test['document group'].map(lambda x: x.split('-')[0])
counts = svm_test['win_doc'].groupby([svm_test['searchterm id']]).value_counts().index.tolist()
test_pred = pd.DataFrame({"searchterm_id":list(x[0] for x in counts),"pre_product_id":list(x[1] for x in counts)}, columns=["searchterm_id","pre_product_id"])

In [18]:
alist =[]
if len(list(set(test_pred['searchterm_id']))) != len(list(set(test['group id']))):
    for a in list(set(test['group id'])):
        if a in list(set(test_pred['searchterm_id'])):
            continue
        alist.append(a)
print(alist)

[49, 375, 390, 1228, 1627, 1665, 1689, 1756, 1906, 1956, 2023, 2388, 2531, 2537, 2578, 2827, 2849, 3169, 3184, 3717, 3750, 4161, 4904, 5510, 5626, 5827, 5848, 5963, 6273, 6351, 7361, 8126, 8175, 9629, 9977, 10415, 10675, 11237, 12306, 12779, 13024, 13552, 13720, 14148, 14149, 14165, 14836, 14999, 15115, 15244, 15617, 15911, 15952]


In [19]:
test[test['group id']==390]

,group id,id,product_uid,relevance,is_query_misspelled,len_of_query,len_of_title,word_in_title_num,word_in_title_sum,word_in_title_let,word_in_title_numratio,word_in_title_letratio,word_in_description_num,word_in_description_sum,word_in_description_let,word_in_description_numratio,word_in_description_letratio,brandinquery_brand_matching,brandinquery_brandintitile_matching,brandinquery_bullets_brand_matching,brandinquery_decription_brand_matching,materialinquery_material_matching,materialinquery_materialintitile_matching,materialinquery_bullets_material_matching,materialinquery_decription_material_matching,word2vec_product_title_1,word2vec_product_desc_1,word2vec_bullet_1,word2vec_all_1,word2vec_product_title_2,word2vec_product_desc_2,word2vec_bullet_2,word2vec_all_2,cv_product_title_sim,cv_product_desc_sim,cv_bullet_sim,tfidf_product_title_sim,tfidf_product_desc_sim,tfidf_bullet_sim,jaccard_product_title,jaccard_product_desc,seqmatch_title_ratio,seqmatch_title_ratioscaled,seqmatch_description_ratio,seqmatch_description_ratioscaled,seqmatch_bullets_ratio,seqmatch_bullets_ratioscaled,seqmatch_desc&bullets_ratio,seqmatch_desc&bullets_ratioscaled
67495,390,148727,155327,3.0,1.0,3,10,3,3,18,1.0,1.0,2,3,13,0.666667,0.722222,0,0,0,0,0,0,0,0,0.838909,0.652061,0.510044,0.666258,0.831388,0.664083,0.548355,0.681199,0.654654,0.143223,0.000000,0.587662,0.293262,0.000000,0.333333,0.041667,0.506329,1.493671,0.029630,0.570370,0.165746,1.334254,0.049383,1.350617
67496,390,12694,102228,3.0,1.0,3,9,3,3,18,1.0,1.0,3,4,18,1.000000,1.000000,0,0,0,0,0,0,0,0,0.823188,0.654660,0.546675,0.679526,0.807674,0.644799,0.547851,0.670067,0.707107,0.255377,0.000000,0.622166,0.382378,0.000000,0.375000,0.060000,0.597015,1.402985,0.034483,0.565518,0.120805,0.779195,0.025105,0.574895
67497,390,113171,137309,3.0,1.0,3,8,3,4,18,1.0,1.0,2,5,13,0.666667,0.722222,0,0,0,0,0,0,0,0,0.960708,0.627982,0.000000,0.701978,0.966690,0.626540,0.000000,0.693567,0.866025,0.141069,0.000000,0.616536,0.365786,0.000000,0.500000,0.031746,0.547945,1.452055,0.023904,0.576096,0.000000,0.000000,0.023857,0.576144
67498,390,82449,124283,3.0,1.0,3,13,3,4,18,1.0,1.0,1,6,8,0.333333,0.444444,0,0,0,0,0,0,0,0,0.862124,0.697136,0.000000,0.738105,0.850827,0.671967,0.000000,0.711611,0.612372,0.000000,0.000000,0.649933,0.452925,0.000000,0.272727,0.014286,0.373832,1.626168,0.009836,0.290165,0.000000,0.000000,0.009820,0.290181
67499,390,81967,124090,3.0,1.0,3,14,3,4,18,1.0,1.0,1,6,8,0.333333,0.444444,0,0,0,0,0,0,0,0,0.849176,0.697136,0.000000,0.738857,0.832411,0.671967,0.000000,0.712005,0.612372,0.000000,0.000000,0.610840,0.452925,0.000000,0.250000,0.014286,0.357143,1.642857,0.009836,0.290165,0.000000,0.000000,0.009820,0.290181
67500,390,124350,142620,3.0,1.0,3,13,3,4,18,1.0,1.0,1,6,8,0.333333,0.444444,0,0,0,0,0,0,0,0,0.856292,0.697136,0.000000,0.737427,0.847000,0.671967,0.000000,0.711441,0.612372,0.000000,0.000000,0.639639,0.452925,0.000000,0.272727,0.014286,0.370370,1.629630,0.009836,0.290165,0.000000,0.000000,0.009820,0.290181
67501,390,95713,129651,3.0,1.0,3,12,3,3,18,1.0,1.0,3,9,18,1.000000,1.000000,0,0,0,0,0,0,0,0,0.811881,0.634305,0.613473,0.655005,0.819316,0.619494,0.607636,0.641278,0.769800,0.419961,0.408248,0.625556,0.428509,0.394316,0.300000,0.037500,0.526316,1.473684,0.015686,0.584315,0.054475,0.645525,0.011964,0.588035
67502,390,58256,115494,3.0,1.0,3,13,3,4,18,1.0,1.0,1,6,8,0.333333,0.444444,0,0,0,0,0,0,0,0,0.851237,0.695901,0.000000,0.737065,0.842596,0.672039,0.000000,0.712703,0.654654,0.000000,0.000000,0.610692,0.451598,0.000000,0.272727,0.014085,0.366972,1.633028,0.009662,0.290337,0.000000,0.000000,0.009646,0.290354


In [20]:
test = test.loc[~test['group id'].isin(alist)]#drop query with constant relevance

In [21]:
test = test.sort_values(['group id','relevance'], ascending=[True, False])

In [22]:
k=10 #average docs per query ~9.69
import ml_metrics as metrics #pip install ml_metrics
Y_p = test.groupby(by='group id', sort=False)['product_uid'].apply( lambda x: x.values.astype(str)[:k].tolist()).values.tolist()
P_p = test_pred.groupby(by='searchterm_id', sort=False).pre_product_id.apply( lambda x: x.values.tolist()).values.tolist()

score = metrics.mapk(Y_p, P_p,k)

print("MAP: %.5f" % score)

MAP: 0.77617
